<a href="https://colab.research.google.com/github/dasdelenbugra/Rag_kullanarak_log_Analizi/blob/main/Rag_kullanarak_log_Analizi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers faiss-cpu transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import numpy as np

# Yanıt boyutunu normal dağılım kullanarak  üretme
def normal_dagilim_boyut_uret(ortalama, std_sapma):
    return max(0, int(np.random.normal(ortalama, std_sapma)))

In [ ]:
# Tarihi ve saati rastgele oluşturma (son 20 gün içinde)
def rastgele_zaman_damgasi_uret():
    bitis_zamani = datetime.datetime.now()
    baslangic_zamani = bitis_zamani - datetime.timedelta(days=20)
    rastgele_zaman = baslangic_zamani + (bitis_zamani - baslangic_zamani) * random.random()
    return rastgele_zaman

In [ ]:
# Apache log formatına uygun sahte log girdisi oluşturma
def sahte_apache_log_girdisi_uret():
    ip_adresi = ".".join(map(str, random.sample(range(1, 256), 4)))
    zaman_damgasi = rastgele_zaman_damgasi_uret()
    istek_turu = random.choice(['GET', 'POST', 'PUT', 'DELETE', 'HEAD', 'OPTIONS', 'PATCH', 'CONNECT', 'TRACE'])
    uri_adresi = random.choice(['/index.html', '/about.html', '/contact.html', '/login', '/products', '/api/data', '/admin'])
    protokol = "HTTP/1.1"
    durum_kodu = random.choice([200, 201, 204, 301, 302, 304, 400, 401, 403, 404, 500, 502, 503, 504])
    boyut = normal_dagilim_boyut_uret(ortalama=1500, std_sapma=500)

    log_girdisi = f'{ip_adresi} - - [{zaman_damgasi.strftime("%d/%b/%Y:%H:%M:%S +0000")}] "{istek_turu} {uri_adresi} {protokol}" {durum_kodu} {boyut}'
    return log_girdisi

In [ ]:
# Belirli bir sayıda sahte log girdisi üretme ve bir dosyaya kaydetme
def sahte_loglari_uret_ve_kaydet(log_sayisi, dosya_adi='sahte_apache_loglari.txt'):
    sahte_loglar = [sahte_apache_log_girdisi_uret() for _ in range(log_sayisi)]

    with open(dosya_adi, 'w') as dosya:
        for girdisi in sahte_loglar:
            dosya.write(girdisi + '\n')


In [ ]:
# Kullanım örneği
sahte_loglari_uret_ve_kaydet(1000)

In [ ]:
import pandas as pd
import re

# Log dosyasını oku ve regex kullanarak ayıkla
log_dosya_yolu = 'sahte_apache_loglari.txt'
log_desen = r'(?P<ip>\d+\.\d+\.\d+\.\d+)\s+-\s+-\s+\[(?P<zaman_damgasi>[^\]]+)\]\s+"(?P<istek_turu>\S+)\s+(?P<uri>\S+)\s+(?P<protokol>\S+)"\s+(?P<durum_kodu>\d+)\s+(?P<boyut>\d+)'


In [ ]:
# Log verilerini bir DataFrame'e yükle
log_df = pd.read_csv(log_dosya_yolu, sep='|', names=['log_girdisi'])
log_df = log_df['log_girdisi'].str.extract(log_desen)

In [ ]:
# Veri türlerini dönüştürme
log_df['zaman_damgasi'] = pd.to_datetime(log_df['zaman_damgasi'], format='%d/%b/%Y:%H:%M:%S +0000')
log_df['durum_kodu'] = log_df['durum_kodu'].astype(int)
log_df['boyut'] = log_df['boyut'].astype(int)

print(log_df.head())

               ip       zaman_damgasi istek_turu            uri  protokol  \
0  21.120.201.146 2024-07-29 06:44:09      TRACE  /contact.html  HTTP/1.1   
1   7.249.148.137 2024-07-30 17:29:10        PUT      /products  HTTP/1.1   
2   21.90.245.124 2024-07-25 19:40:59        GET    /about.html  HTTP/1.1   
3   196.67.164.31 2024-07-28 10:19:53      TRACE         /login  HTTP/1.1   
4   230.22.237.17 2024-07-26 02:00:55        PUT  /contact.html  HTTP/1.1   

   durum_kodu  boyut  
0         401    668  
1         504   1727  
2         304   1233  
3         301    783  
4         404    567  


In [ ]:
# Geçersiz durum kodları veya boyut değerlerini kontrol etme
gecersiz_durumlar = log_df[~log_df['durum_kodu'].between(100, 599)]  # HTTP durum kodları 100-599 aralığındadır
gecersiz_boyutlar = log_df[log_df['boyut'] < 0]  # Boyut negatif olmamalıdır

print(f"Geçersiz durum kayıtları: {len(gecersiz_durumlar)}")
print(f"Geçersiz boyut kayıtları: {len(gecersiz_boyutlar)}")


Geçersiz durum kayıtları: 0
Geçersiz boyut kayıtları: 0


In [ ]:
# En sık kullanılan durum kodlarını analiz etme
durum_kodu_sayilari = log_df['durum_kodu'].value_counts()
print(durum_kodu_sayilari)

durum_kodu
400    85
301    81
401    80
502    78
302    78
503    74
404    73
500    71
201    70
204    67
504    64
304    63
403    61
200    55
Name: count, dtype: int64


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline
import numpy as np

# Model ve FAISS kurulumları
model = SentenceTransformer('all-MiniLM-L6-v2')
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Log verilerini birleştirerek metin formatında kullanıma hazırlama
log_df['log_metni'] = log_df.apply(lambda row: f"{row['istek_turu']} {row['uri']} {row['durum_kodu']}", axis=1)

In [ ]:
# Metinleri vektörlere dönüştürme
log_vektorleri = model.encode(log_df['log_metni'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
# FAISS kullanarak vektörleri bir veri tabanında depolama
indeks = faiss.IndexFlatL2(log_vektorleri.shape[1])  # L2 mesafe metriği
indeks.add(np.array(log_vektorleri, dtype='float32'))

def ilgili_loglari_getir(sorgu):
    sorgu_vektoru = model.encode([sorgu], show_progress_bar=False)
    mesafeler, indeksler = indeks.search(np.array(sorgu_vektoru, dtype='float32'), k=4)
    return log_df.iloc[indeksler[0]]

def yanit_uret(sorgu, ilgili_loglar):
    if ilgili_loglar.empty:
        return "Benzer log girdisi bulunamadı."

    baglam = "\n".join([
        f"{row['zaman_damgasi']} tarihinde {row['istek_turu']} isteği {row['uri']} adresine yapılmış ve {row['durum_kodu']} durum kodu ile sonuçlanmış."
        for _, row in ilgili_loglar.iterrows()
    ])

    sonuc = qa_pipeline(
        question=sorgu,
        context=baglam,
        max_length=150,
        num_beams=5,
        early_stopping=True
    )

    return sonuc.get('answer', 'Yanıt üretilemedi.')

def main_function():
    while True:
        sorgu = input("Sorgunuzu girin ('q' ile çıkış yapın): ")
        if sorgu.lower() == 'q':
            break

        ilgili_loglar = ilgili_loglari_getir(sorgu)
        yanit = yanit_uret(sorgu, ilgili_loglar)

        print("\nSorgu:", sorgu)
        if not ilgili_loglar.empty:
            print("İlgili Loglar:\n", ilgili_loglar.to_string(index=False))
        else:
            print("Benzer log bulunamadı.")
        print("Yanıt:", yanit)

if __name__ == "__main__":
    main_function()

**Proje Raporu: Apache Log Verisi ile Soru-Cevap Sistemi Geliştirme**

**Amaç ve Kapsam**

Bu projede, bir web sitesi için oluşturulan trafik loglarını kullanarak bir soru-cevap (Q&A) sistemi geliştirilmek istenmektedir. Bu sistem, kullanıcılardan gelen doğal dildeki soruları alacak, ilgili log verilerini analiz edecek ve bu verilerle en uygun yanıtları oluşturacaktır. Sistemin temelinde Retrieval-Augmented Generation (RAG) modeli yer alacaktır.

1. **Proje Adımları**

**Adım 1: Veri Üretimi ve Hazırlığı**

Projeye başlarken, Apache log formatına uygun sahte log verileri oluşturmak için bir Python fonksiyonu geliştirdim. Bu fonksiyon, rastgele IP adresleri, HTTP istek türleri, URI'ler, durum kodları ve boyut bilgileri oluşturuyor. Log verilerinin zaman damgası kısmı, son 30 gün içerisindeki rastgele bir tarihten seçiliyor. Sahte veriler, gerçeğe yakın Apache log girdileri üretmek amacıyla normal dağılım kullanılarak boyut verileri üretiyor ve rastgele seçilen HTTP durum kodları ile log formatına uygun hale getiriliyor.

**Adım 2: Veri İşleme ve Temizleme**

Sahte log verileri oluşturulduktan sonra, bu verileri işlemek için bir veri temizleme ve ayrıştırma süreci uygulandı. Regex (düzenli ifade) kullanarak log dosyasındaki IP adresi, zaman damgası, istek türü, URI, protokol, durum kodu ve boyut bilgileri ayrıştırıldı ve bir pandas DataFrame'ine yüklendi. Zaman damgaları datetime formatına, durum kodları ve boyutlar ise tamsayı formatına dönüştürüldü. Bu aşamada veri türleri doğru şekilde ayarlandı ve log dosyalarının tutarlı olması sağlandı.

**Adım 3: Model Entegrasyonu**

Projenin ana bileşenlerinden biri, Sentence Transformer modeli kullanarak log verilerinin vektörlere dönüştürülmesiydi. Vektörlere dönüştürülen log verileri, FAISS kütüphanesi kullanılarak bir vektör veri tabanına yüklendi. Bu aşamada, log verilerinden anlamlı vektörler elde edilmesi ve sorgulara benzer logların bulunması sağlandı.

Ayrıca, BERT tabanlı bir soru-cevap modeli (pipeline) entegre edilerek kullanıcı sorgularına uygun yanıtlar üretilmeye çalışıldı. generate_answer fonksiyonu ile, elde edilen benzer log girdileri kullanılarak kullanıcı sorularına yanıt üretildi.

**Adım 4: Performans Değerlendirmesi ve Sonuçlar**

Sistemin performansını değerlendirmek için farklı sorgular denendi ve ilgili log girdilerinin doğru şekilde bulunup bulunmadığı analiz edildi. Sistemin doğruluğu, üretilen yanıtların kalitesi ve benzer logların doğru seçilip seçilmediği kriterlerine göre değerlendirildi.

**2. Karşılaşılan Zorluklar**

**Veri Ayrıştırma Zorlukları**

Regex kullanarak log verilerini ayrıştırmak her zaman düzgün sonuçlar vermeyebilir. Özellikle log formatında varyasyonlar olduğunda veya eksik veriler bulunduğunda regex deseninin tüm verileri doğru şekilde ayrıştıramadığını gözlemledim. Bu durum, eksik veya hatalı veri türü dönüşümlerine neden olabilir. Bu nedenle, veri temizleme adımlarını daha dikkatli ve kapsamlı yapmak gerekti.

**Model Performansı**

Sentence Transformer modeli, metinleri anlamlı vektörlere dönüştürmede oldukça başarılı, ancak bazı durumlarda log verilerinin içerdiği bilgilerin sınırlı olması nedeniyle modelin ürettiği vektörler benzer sorgulara yeterince iyi yanıt veremeyebiliyor. Bunun yanı sıra, FAISS kütüphanesi ile yapılan aramalarda bazen beklenmeyen sonuçlar elde edilebiliyor, bu da modelin ve vektör veri tabanının daha fazla optimizasyon gerektirdiğini gösteriyor.

Yanıt Üretimi
BERT tabanlı soru-cevap modeli, bağlam sağlandığında anlamlı yanıtlar üretebiliyor. Ancak, sağlanan bağlam sınırlı veya yetersiz olduğunda, modelin ürettiği yanıtlar tutarsız veya alakasız olabiliyor. Bu durum, bağlamın genişletilmesi ve yanıt üretim sürecinin iyileştirilmesi gerektiğini gösteriyor.

**3. Performans ve Doğruluk Değerlendirmesi**

Sistemin performansı genel olarak tatmin edici, ancak bazı alanlarda iyileştirme yapılması gerektiği aşikar. Sistemin doğru log girdilerini bulma yeteneği, FAISS ve model entegrasyonunun başarılı bir şekilde yapılmış olmasıyla doğru orantılı. Ancak, aşağıdaki alanlarda iyileştirmeler yapılabilir:

Daha İyi Veri Temizleme: Sistemin doğruluğunu artırmak için daha kapsamlı bir veri temizleme süreci uygulanmalı ve eksik veya hatalı log girdileri otomatik olarak tespit edilip işlenmelidir.

Vektörlerin ve FAISS Optimizasyonu: FAISS'in vektör tabanlı aramalarındaki performansı artırılabilir ve kullanılan vektörlerin boyutu daha verimli hale getirilebilir.

Yanıt Üretme Sürecinin Geliştirilmesi: Soru-cevap modelinin bağlamı daha iyi kavrayabilmesi için bağlamın genişletilmesi ve daha fazla veri sağlanması düşünülebilir. Ayrıca, farklı modellerle karşılaştırma yaparak daha kaliteli yanıtlar elde edilebilir.

**4. İyileştirme Önerileri**

Regex Deseninin Genişletilmesi: Log verilerinin farklı formatlara uygun olarak ayrıştırılması için regex desenini genişletmek faydalı olacaktır.

FAISS için Cosine Similarity Kullanımı: L2 mesafe metriği yerine cosine similarity gibi başka bir mesafe metriği denenebilir, bu da özellikle metin tabanlı vektörlerde daha iyi sonuçlar verebilir.

Daha Fazla Veri Sağlama: Yanıt üretim sürecinde modelin daha fazla bağlam kullanmasını sağlamak için daha fazla log girdisi sağlanabilir. Bu, modelin yanıtlarının kalitesini artırabilir.

Query Preprocessing: Kullanıcı sorgularının daha iyi analiz edilmesi ve işlenmesi için ek doğal dil işleme teknikleri entegre edilebilir.